In [53]:
!pip freeze | grep scikit-learn

python(22093) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


scikit-learn==1.4.1.post1


In [54]:
!python -V

python(22161) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


Python 3.11.2


In [55]:
import pickle
import pandas as pd
from sklearn.metrics import mean_squared_error
import numpy as np
import os

In [56]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

/Users/karoryba/.pyenv/versions/3.11.2/lib/python3.11/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator DictVectorizer from version 1.5.0 when using version 1.4.1.post1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/karoryba/.pyenv/versions/3.11.2/lib/python3.11/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LinearRegression from version 1.5.0 when using version 1.4.1.post1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [57]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [58]:
df = read_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-05.parquet')

In [64]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_val = df.duration.values
y_pred = model.predict(X_val)

std_dev = np.std(y_pred)
print(std_dev)

mean_duration = np.mean(y_pred)

print(f"Mean predicted duration: {mean_duration:.2f} minutes")

6.263976619625352
Mean predicted duration: 14.24 minutes


In [60]:
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [61]:
def apply_model(input_file, run_id, output_file):
    df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')
    df_result = pd.DataFrame()
    # df_result['ride_id'] = df['ride_id']
    # # df_result['lpep_pickup_datetime'] = df['lpep_pickup_datetime']
    # df_result['PULocationID'] = df['PULocationID']
    # df_result['DOLocationID'] = df['DOLocationID']
    # df_result['actual_duration'] = df['duration']
    # df_result['predicted_duration'] = y_pred
    # df_result['diff'] = df_result['actual_duration'] - df_result['predicted_duration']
    # df_result['model_version'] = run_id
    
    df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

In [62]:
year = 2023
month = 3
taxi_type = 'green'

input_file = f'https://d37ci6vzurychx.cloudfront.net/trip-data/{taxi_type}_tripdata_{year:04d}-{month:02d}.parquet'
output_file = f'output/{taxi_type}/{year:04d}-{month:02d}.parquet'

RUN_ID = os.getenv('RUN_ID', 'e1efc53e9bd149078b0c12aeaa6365df')

In [63]:
apply_model(input_file=input_file, run_id=RUN_ID, output_file=output_file)